In [186]:
## Import libraries
import numpy as np
import pandas as pd
import os

In [187]:
## Set working directory
os.chdir('E:/CR2/Repos/TNC-Demand-Model-Southeast/')

In [188]:
### Calculating Utility
## Get dataframe of predicted trips
utility = pd.read_csv('Cleaned_Inputs/Intermediate/pred_trips.csv')
utility

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES
0,21067003802,21067004007,40.016667,20.158829,40.016667,44.016667,31.260439,20.105323
1,21067004007,21067003802,36.616667,24.871705,36.616667,40.616667,34.434858,23.364816
2,21067004007,21067003701,40.533333,22.877503,40.533333,44.533333,33.808273,22.134277
3,21067003701,21067004007,38.750000,22.608280,38.750000,42.750000,33.055025,21.829425
4,21067004207,21067003918,32.583333,19.840966,32.583333,36.583333,28.830300,19.425726
...,...,...,...,...,...,...,...,...
6719,21067001100,21067001100,0.000000,0.000000,0.000000,4.000000,1.903500,2.889600
6720,21067001300,21067001300,0.000000,0.000000,0.000000,4.000000,1.903500,2.889600
6721,21067003406,21067003406,0.000000,0.000000,0.000000,4.000000,1.903500,2.889600
6722,21067004003,21067004003,0.000000,0.000000,0.000000,4.000000,1.903500,2.889600


In [189]:
## Create indicator to see if an airport is in the origin or destination
airport_census_tract = 21067004207
utility['airport'] = np.where(utility['origin'] == airport_census_tract, 1, 
                              np.where(utility['destination'] == airport_census_tract, 1, 0))
utility.loc[utility['airport'] == 1].head()

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES,airport
4,21067004207,21067003918,32.583333,19.840966,32.583333,36.583333,28.830300,19.425726,1
5,21067003918,21067004207,34.933333,18.640822,34.933333,38.933333,28.451153,18.684740,1
20,21067004207,21067004007,36.800000,21.635254,36.800000,40.800000,31.632681,20.997348,1
21,21067004007,21067004207,39.600000,22.305746,39.600000,43.600000,33.033950,21.658093,1
34,21067003802,21067004207,28.566667,16.904638,28.566667,32.566667,25.078252,17.026818,1


In [190]:
## Calculate utility for Private Trips
## Recall: utility for private trips is a function of travel time and cost.
utility.loc[:, 'PRIVATE_UTILITY'] = (-0.110 * utility['PRIVATE_TRAVEL_TIME']) + (-0.138 * utility['PRIVATE_TRIP_FARES'])
utility.head(7)

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES,airport,PRIVATE_UTILITY
0,21067003802,21067004007,40.016667,20.158829,40.016667,44.016667,31.260439,20.105323,0,-8.715774
1,21067004007,21067003802,36.616667,24.871705,36.616667,40.616667,34.434858,23.364816,0,-8.779844
2,21067004007,21067003701,40.533333,22.877503,40.533333,44.533333,33.808273,22.134277,0,-9.124208
3,21067003701,21067004007,38.750000,22.608280,38.750000,42.750000,33.055025,21.829425,0,-8.824093
4,21067004207,21067003918,32.583333,19.840966,32.583333,36.583333,28.830300,19.425726,1,-7.562748
5,21067003918,21067004207,34.933333,18.640822,34.933333,38.933333,28.451153,18.684740,1,-7.768926
6,21067003703,21067004007,34.466667,24.226410,34.466667,38.466667,33.243799,22.761588,0,-8.378978


In [191]:
#### Calculate utility for shared trips
## Recall: utility for shared trips is a function of travel time, travel cost, median income in the origin, 
## median income in the destination, and if it's going to or from the airport.

#### Read in income file
median_inc = pd.read_csv('Raw_Inputs/ACS/S1903/S1903 Data.csv')
median_inc[['TRACT', 'COUNTY', 'STATE']] = median_inc.NAME.str.split(',', expand=True)
median_inc.head()

,GEO_ID,NAME,S1903_C01_001E,S1903_C01_001M,S1903_C01_002E,S1903_C01_002M,S1903_C01_003E,S1903_C01_003M,S1903_C01_004E,S1903_C01_004M,...,S1903_C03_038E,S1903_C03_038M,S1903_C03_039E,S1903_C03_039M,S1903_C03_040E,S1903_C03_040M,Unnamed: 242,TRACT,COUNTY,STATE
0,Geography,Geographic Area Name,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND...,Margin of Error!!Number!!HOUSEHOLD INCOME BY R...,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND...,Margin of Error!!Number!!HOUSEHOLD INCOME BY R...,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND...,Margin of Error!!Number!!HOUSEHOLD INCOME BY R...,Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND...,Margin of Error!!Number!!HOUSEHOLD INCOME BY R...,...,Estimate!!Median income (dollars)!!NONFAMILY H...,Margin of Error!!Median income (dollars)!!NONF...,Estimate!!Median income (dollars)!!NONFAMILY H...,Margin of Error!!Median income (dollars)!!NONF...,Estimate!!Median income (dollars)!!NONFAMILY H...,Margin of Error!!Median income (dollars)!!NONF...,NaN,Geographic Area Name,None,None
1,1400000US21067000101,"Census Tract 1.01, Fayette County, Kentucky",1993,120,1475,123,433,93,0,11,...,19639,10162,17043,2992,35739,4356,NaN,Census Tract 1.01,Fayette County,Kentucky
2,1400000US21067000102,"Census Tract 1.02, Fayette County, Kentucky",984,108,646,97,268,69,0,11,...,33021,7825,26719,13232,72411,29957,NaN,Census Tract 1.02,Fayette County,Kentucky
3,1400000US21067000200,"Census Tract 2, Fayette County, Kentucky",1225,128,648,139,503,129,0,11,...,27273,15009,23864,8889,41198,3724,NaN,Census Tract 2,Fayette County,Kentucky
4,1400000US21067000300,"Census Tract 3, Fayette County, Kentucky",1249,99,573,104,585,98,0,11,...,38920,20635,-,**,50972,21208,NaN,Census Tract 3,Fayette County,Kentucky


In [192]:
# Filter for Fayette County
median_inc = median_inc[median_inc['COUNTY'] == ' Fayette County']
# Get census tract from GEO ID
median_inc['CENSUS_TRACT'] = median_inc.apply(lambda row: row['GEO_ID'][9:], axis = 1)
# Rename column and filter
median_inc = median_inc.rename(columns={'S1903_C03_001E': 'MEDIAN_INC'})
median_inc = median_inc[['CENSUS_TRACT', 'MEDIAN_INC']]
median_inc['MEDIAN_INC'] = median_inc.MEDIAN_INC.astype(float)
median_inc['MEDIAN_INC'] = median_inc['MEDIAN_INC']/10000
median_inc.head()

,CENSUS_TRACT,MEDIAN_INC
1,21067000101,2.5363
2,21067000102,2.6477
3,21067000200,2.6725
4,21067000300,3.2089
5,21067000400,2.7031


In [193]:
### Merge utility dataframe with income dataframe on census tract (by origin)
utility = utility.merge(median_inc.astype(float), how='left', left_on='origin', right_on='CENSUS_TRACT')
## Rename to column that indicates median income of the origin
utility.rename(columns={'MEDIAN_INC': 'ORIGIN_MEDIAN_INC'}, inplace=True)
utility.drop(columns='CENSUS_TRACT', inplace=True)
utility.head()

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES,airport,PRIVATE_UTILITY,ORIGIN_MEDIAN_INC
0,21067003802,21067004007,40.016667,20.158829,40.016667,44.016667,31.260439,20.105323,0,-8.715774,6.7212
1,21067004007,21067003802,36.616667,24.871705,36.616667,40.616667,34.434858,23.364816,0,-8.779844,8.1250
2,21067004007,21067003701,40.533333,22.877503,40.533333,44.533333,33.808273,22.134277,0,-9.124208,8.1250
3,21067003701,21067004007,38.750000,22.608280,38.750000,42.750000,33.055025,21.829425,0,-8.824093,9.3750
4,21067004207,21067003918,32.583333,19.840966,32.583333,36.583333,28.830300,19.425726,1,-7.562748,12.7798


In [194]:
### Merge utility dataframe with income dataframe on census tract (by destination)
utility = utility.merge(median_inc.astype(float), how='left', left_on='destination', right_on='CENSUS_TRACT')
## Rename to column that indicates median income of the origin
utility.rename(columns={'MEDIAN_INC': 'DESTINATION_MEDIAN_INC'}, inplace=True)
utility.drop(columns='CENSUS_TRACT', inplace=True)
utility.head()

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES,airport,PRIVATE_UTILITY,ORIGIN_MEDIAN_INC,DESTINATION_MEDIAN_INC
0,21067003802,21067004007,40.016667,20.158829,40.016667,44.016667,31.260439,20.105323,0,-8.715774,6.7212,8.1250
1,21067004007,21067003802,36.616667,24.871705,36.616667,40.616667,34.434858,23.364816,0,-8.779844,8.1250,6.7212
2,21067004007,21067003701,40.533333,22.877503,40.533333,44.533333,33.808273,22.134277,0,-9.124208,8.1250,9.3750
3,21067003701,21067004007,38.750000,22.608280,38.750000,42.750000,33.055025,21.829425,0,-8.824093,9.3750,8.1250
4,21067004207,21067003918,32.583333,19.840966,32.583333,36.583333,28.830300,19.425726,1,-7.562748,12.7798,8.7656


In [195]:
### Calculate Utility for Shared Trips
utility.loc[:, 'SHARED_UTILITY'] = (-0.573223) + (-0.110075 * utility['SHARED_TRAVEL_TIME']) + (-0.137794 * utility['SHARED_TRIP_FARES'])  + (-0.068228 * utility['ORIGIN_MEDIAN_INC'])  + (-0.049448 * utility['DESTINATION_MEDIAN_INC']) + (-2.716947 * utility['airport'])
utility.head()

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES,airport,PRIVATE_UTILITY,ORIGIN_MEDIAN_INC,DESTINATION_MEDIAN_INC,SHARED_UTILITY
0,21067003802,21067004007,40.016667,20.158829,40.016667,44.016667,31.260439,20.105323,0,-8.715774,6.7212,8.1250,-9.049090
1,21067004007,21067003802,36.616667,24.871705,36.616667,40.616667,34.434858,23.364816,0,-8.779844,8.1250,6.7212,-9.150336
2,21067004007,21067003701,40.533333,22.877503,40.533333,44.533333,33.808273,22.134277,0,-9.124208,8.1250,9.3750,-9.543128
3,21067003701,21067004007,38.750000,22.608280,38.750000,42.750000,33.055025,21.829425,0,-8.824093,9.3750,8.1250,-9.328296
4,21067004207,21067003918,32.583333,19.840966,32.583333,36.583333,28.830300,19.425726,1,-7.562748,12.7798,8.7656,-11.299210


In [196]:
### Calculate Probability of Taking a Private/Shared Ride between an OD Pair
## P(Private) = e^(U(Private)) / sum(e^(U(Private))+e^(U(Shared))
utility.loc[:, 'PRIV_PROB'] = (np.exp(utility['PRIVATE_UTILITY'])) / ((np.exp(utility['PRIVATE_UTILITY'])) + (np.exp(utility['SHARED_UTILITY'])))
utility.loc[:, 'SHARED_PROB'] = 1 - utility['PRIV_PROB']
utility.sort_values(by=['PRIV_PROB'])

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES,airport,PRIVATE_UTILITY,ORIGIN_MEDIAN_INC,DESTINATION_MEDIAN_INC,SHARED_UTILITY,PRIV_PROB,SHARED_PROB
68,21067002001,21067004007,36.666667,15.408888,36.666667,40.666667,26.101221,16.413592,0,-7.635302,3.0807,8.1250,-7.923256,0.571495,0.428505
0,21067003802,21067004007,40.016667,20.158829,40.016667,44.016667,31.260439,20.105323,0,-8.715774,6.7212,8.1250,-9.049090,0.582566,0.417434
1,21067004007,21067003802,36.616667,24.871705,36.616667,40.616667,34.434858,23.364816,0,-8.779844,8.1250,6.7212,-9.150336,0.591578,0.408422
126,21067001900,21067004007,31.783333,15.158152,31.783333,35.783333,24.467762,15.936326,0,-6.872718,2.5868,8.1250,-7.286261,0.601937,0.398063
2,21067004007,21067003701,40.533333,22.877503,40.533333,44.533333,33.808273,22.134277,0,-9.124208,8.1250,9.3750,-9.543128,0.603225,0.396775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5138,21067004207,21067004205,12.150000,4.291293,12.150000,16.150000,9.203059,6.772271,1,-2.606522,12.7798,9.7410,-7.354673,0.991407,0.008593
5434,21067004208,21067004207,10.966667,4.354002,10.966667,14.966667,8.916161,6.747356,1,-2.436764,10.2478,12.7798,-7.198493,0.991522,0.008478
5435,21067004207,21067004208,11.133333,4.360545,11.133333,15.133333,8.970133,6.762165,1,-2.462545,12.7798,10.2478,-7.266431,0.991869,0.008131
423,21067004006,21067004207,28.866667,11.945562,28.866667,32.866667,20.790114,13.400393,1,-6.044369,22.8125,12.7798,-10.942849,0.992597,0.007403


In [197]:
### Calculate Mode Choice Logsums
utility['MODE_LOGSUM'] = np.log((np.exp(utility['SHARED_UTILITY'])) + (np.exp(utility['PRIVATE_UTILITY'])))
utility.head()

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES,airport,PRIVATE_UTILITY,ORIGIN_MEDIAN_INC,DESTINATION_MEDIAN_INC,SHARED_UTILITY,PRIV_PROB,SHARED_PROB,MODE_LOGSUM
0,21067003802,21067004007,40.016667,20.158829,40.016667,44.016667,31.260439,20.105323,0,-8.715774,6.7212,8.1250,-9.049090,0.582566,0.417434,-8.175461
1,21067004007,21067003802,36.616667,24.871705,36.616667,40.616667,34.434858,23.364816,0,-8.779844,8.1250,6.7212,-9.150336,0.591578,0.408422,-8.254882
2,21067004007,21067003701,40.533333,22.877503,40.533333,44.533333,33.808273,22.134277,0,-9.124208,8.1250,9.3750,-9.543128,0.603225,0.396775,-8.618743
3,21067003701,21067004007,38.750000,22.608280,38.750000,42.750000,33.055025,21.829425,0,-8.824093,9.3750,8.1250,-9.328296,0.623446,0.376554,-8.351601
4,21067004207,21067003918,32.583333,19.840966,32.583333,36.583333,28.830300,19.425726,1,-7.562748,12.7798,8.7656,-11.299210,0.976717,0.023283,-7.539189


In [199]:
### Used in Destination Choice: Break out utilities by whether they are airport trips or not.
utility['mode_logsum_airport'] = utility['MODE_LOGSUM'] * utility['airport']
utility['mode_logsum_no_airport'] = np.where(utility['airport'] == 0, utility['MODE_LOGSUM'], 0)
utility.head()

,origin,destination,travel_time_min,distance_miles,PRIVATE_TRAVEL_TIME,SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRIP_FARES,airport,PRIVATE_UTILITY,ORIGIN_MEDIAN_INC,DESTINATION_MEDIAN_INC,SHARED_UTILITY,PRIV_PROB,SHARED_PROB,MODE_LOGSUM,mode_logsum_airport,mode_logsum_no_airport
0,21067003802,21067004007,40.016667,20.158829,40.016667,44.016667,31.260439,20.105323,0,-8.715774,6.7212,8.1250,-9.049090,0.582566,0.417434,-8.175461,-0.000000,-8.175461
1,21067004007,21067003802,36.616667,24.871705,36.616667,40.616667,34.434858,23.364816,0,-8.779844,8.1250,6.7212,-9.150336,0.591578,0.408422,-8.254882,-0.000000,-8.254882
2,21067004007,21067003701,40.533333,22.877503,40.533333,44.533333,33.808273,22.134277,0,-9.124208,8.1250,9.3750,-9.543128,0.603225,0.396775,-8.618743,-0.000000,-8.618743
3,21067003701,21067004007,38.750000,22.608280,38.750000,42.750000,33.055025,21.829425,0,-8.824093,9.3750,8.1250,-9.328296,0.623446,0.376554,-8.351601,-0.000000,-8.351601
4,21067004207,21067003918,32.583333,19.840966,32.583333,36.583333,28.830300,19.425726,1,-7.562748,12.7798,8.7656,-11.299210,0.976717,0.023283,-7.539189,-7.539189,0.000000


In [200]:
### Write to CSV
mode_choice_utility = utility
mode_choice_utility.to_csv('Cleaned_Inputs/Intermediate/mode_choice_utility_output.csv')